In [1]:
import pandas as pd
import numpy as no

In [3]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')
resources=pd.read_csv('data/resources.csv')


/home/ubuntu/anaconda/envs/kaggle/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(train.dtypes)
print(test.dtypes)
print(resources.dtypes)

id                                              object
teacher_id                                      object
teacher_prefix                                  object
school_state                                    object
project_submitted_datetime                      object
project_grade_category                          object
project_subject_categories                      object
project_subject_subcategories                   object
project_title                                   object
project_essay_1                                 object
project_essay_2                                 object
project_essay_3                                 object
project_essay_4                                 object
project_resource_summary                        object
teacher_number_of_previously_posted_projects     int64
project_is_approved                              int64
dtype: object
id                                              object
teacher_id                                      obj

In [12]:
tid=['p036502']

In [14]:
train[train['id'].isin(tid)]

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1


In [15]:
resources['id'].isin(tid)

0          p233245
1          p069063
2          p069063
3          p069063
4          p069063
5          p069063
6          p069063
7          p069063
8          p096795
9          p096795
10         p096795
11         p149007
12         p149007
13         p149007
14         p236235
15         p052460
16         p052460
17         p052460
18         p052460
19         p233680
20         p233680
21         p233680
22         p233680
23         p233680
24         p233680
25         p233680
26         p233680
27         p233680
28         p233680
29         p233680
            ...   
1541242    p187432
1541243    p187432
1541244    p187432
1541245    p149426
1541246    p238803
1541247    p087783
1541248    p087783
1541249    p087783
1541250    p087783
1541251    p087783
1541252    p087783
1541253    p087783
1541254    p086116
1541255    p086116
1541256    p086116
1541257    p086116
1541258    p086116
1541259    p086116
1541260    p228679
1541261    p228679
1541262    p228679
1541263    p